In [1]:
import pandas as pd
import numpy as np
import re
import json
import matplotlib.pyplot as plt
import glob, os, csv

from pprint import pprint
from langdetect import detect

import gensim
from gensim.utils import simple_preprocess
from gensim import corpora, models
from gensim.models import CoherenceModel

from bs4 import BeautifulSoup

import ast
from collections import Counter
from operator import add

from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer

from sklearn.model_selection import train_test_split
import math
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction import DictVectorizer
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import VotingClassifier, StackingClassifier, AdaBoostClassifier
import seaborn as sns
from sklearn.metrics import classification_report
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix

import nltk
# nltk.download('stopwords')
# nltk.download('wordnet')
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
nltk.download('words')
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.corpus import words
from nltk.corpus import wordnet
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'http', 'javascript'])

pd.options.mode.chained_assignment = None

[nltk_data] Downloading package words to /home/justin/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [2]:
## Categories ##
# 1: Market: Drug, gun, 
# 2: Counterfeit: counterfeit credit cards, money, ID
# 3: Services: Hosting service, forum, email, pastebin, file-sharing
# 4: Security: Security-related information, tutorials or services, leaked data
# 5: Porn: Hosting pornographic material
# 6: Cryptocurrency
# 7: NoAccess: Login, Down, Empty
# 8: Other: Cannot be classified in any other category (e.g. personal blog)

In [3]:
# Dictionary of conversion of categories
conversion_dict = {
    'Art': 'Other',
    'Casino': 'Services',
    'Counterfeit Credit-Cards': 'Counterfeit',
    'Counterfeit Money': 'Counterfeit',
    'Counterfeit Personal-Identification': 'Counterfeit',
    'Cryptocurrency': 'Cryptocurrency',
    'Cryptolocker': 'Security',
    'Down': 'NoAccess',
    'Drugs': 'Market',
    'Empty': 'NoAccess',
    'Forum': 'Services',
    'Hacking': 'Security',
    'Hosting': 'Services',
    'Leaked-Data': 'Security',
    'Library': 'Other',
    'Locked': 'NoAccess',
    'Marketplace': 'Market',
#     'Onion Directory/Wiki': ['directory', 'dir', 'wiki'],
    'Personal': 'Other',
    'Politics': 'Other',
    'Porno': 'Porn',
    'Religion': 'Other',
    'Services': 'Services',
    'Social-Network': 'Services',
    'Violence': 'Market'
}

In [4]:
def get_domain(url):
    url = str(url)
    result = re.findall('^(?:https?\:\/\/)?[\w\-\.]+\.onion', url)
    if len(result) < 1:
        return 'NA'
    else:
        return result[0]

In [5]:
# Read input data
df_orig = pd.read_csv('../data/model_training_dataset.csv')
df_orig['url'] = df_orig['url'].apply(ast.literal_eval)
df_orig['body_text'] = df_orig['body_text'].apply(ast.literal_eval)
df_orig['body_token'] = df_orig['body_token'].apply(ast.literal_eval)
df_orig['class'] = df_orig['class'].apply(lambda x: x.strip())
df_orig

,url,title,body_text,body_token,language,class
0,[wwjewp6ca4rkudyj.onion],\n Frënn vun der Ënn - Index\n,[support FVDE because even small organizations...,"[support, fvde, becaus, even, small, organ, co...",en,Services
1,[u76xgym22s4adf55.onion],PayPal Master: Home,[Questions Telegram us at httpsme ppmasteroffi...,"[question, telegram, u, httpsme, avail, respon...",en,Counterfeit
2,[2hftxvyft7dl3fk2.onion],Umbrella Escrow,"[View existing escrow, Buyer starts an escrow ...","[view, exist, escrow, buyer, start, escrow, se...",en,Cryptocurrency
3,[elherbotsiddarol.onion],ElHerbolario,"[Thanks, Fran ais Tutoriel, Listings price dro...","[thank, fran, ai, tutoriel, list, price, drop,...",en,Market
4,[yermrrzeg4fusqx5.onion],anonymous OnionMail Server,"[To anonymous, anonymous OnionMail Server, Inf...","[anonym, anonym, onionmail, server, info, encr...",en,Services
...,...,...,...,...,...,...
1981,[5wkkhgqtity7bhdf.onion],walletGenerator - Universal Paper wallet gener...,"[Firefox 21 may 2013, English, Add support for...","[firefox, may, english, add, support, alqo, co...",en,Cryptocurrency
1982,[ctzzqqimlfamyhrc.onion],| TheYOSH.nl,[The last month the livecounter theyosh nl web...,"[last, month, livecount, theyosh, nl, websit, ...",en,Services
1983,[grrmailb3fxpjbwm.onion],✉ Guerrilla Mail on Tor,"[Your Email Address is, Javascript is disabled...","[email, address, disabl, featur, avail, thank,...",en,Services
1984,[greenegbqkyk3ois.onion],✔ Legit Carding Services ♛ Money Transfer Worl...,"[AmericanExpress PrePaid Cards, Hacked US Mast...","[americanexpress, prepaid, card, hack, u, mast...",en,Counterfeit


In [6]:
data = []
for i, row in df_orig.iterrows():
    for url in row['url']:
        temp_row = [url, row['title'], row['body_text'], row['body_token'], row['language'], row['class']]
        data.append(temp_row)
df_split = pd.DataFrame(data)
df_split.rename(columns={0: 'domain_url', 1: 'title', 2: 'body_text', 3: 'body_token', 4: 'language', 5: 'class'}, inplace=True)
df_split

,domain_url,title,body_text,body_token,language,class
0,wwjewp6ca4rkudyj.onion,\n Frënn vun der Ënn - Index\n,[support FVDE because even small organizations...,"[support, fvde, becaus, even, small, organ, co...",en,Services
1,u76xgym22s4adf55.onion,PayPal Master: Home,[Questions Telegram us at httpsme ppmasteroffi...,"[question, telegram, u, httpsme, avail, respon...",en,Counterfeit
2,2hftxvyft7dl3fk2.onion,Umbrella Escrow,"[View existing escrow, Buyer starts an escrow ...","[view, exist, escrow, buyer, start, escrow, se...",en,Cryptocurrency
3,elherbotsiddarol.onion,ElHerbolario,"[Thanks, Fran ais Tutoriel, Listings price dro...","[thank, fran, ai, tutoriel, list, price, drop,...",en,Market
4,yermrrzeg4fusqx5.onion,anonymous OnionMail Server,"[To anonymous, anonymous OnionMail Server, Inf...","[anonym, anonym, onionmail, server, info, encr...",en,Services
...,...,...,...,...,...,...
10801,ctzzqqimlfamyhrc.onion,| TheYOSH.nl,[The last month the livecounter theyosh nl web...,"[last, month, livecount, theyosh, nl, websit, ...",en,Services
10802,grrmailb3fxpjbwm.onion,✉ Guerrilla Mail on Tor,"[Your Email Address is, Javascript is disabled...","[email, address, disabl, featur, avail, thank,...",en,Services
10803,greenegbqkyk3ois.onion,✔ Legit Carding Services ♛ Money Transfer Worl...,"[AmericanExpress PrePaid Cards, Hacked US Mast...","[americanexpress, prepaid, card, hack, u, mast...",en,Counterfeit
10804,mldozg3tw4qyzavm.onion,��� ����� � ������� - The Front Page,"[onion Tor Browser, CMS, amazing corn fields 1...","[onion, tor, browser, cm, amaz, corn, field, j...",en,NoAccess


In [7]:
df_split['domain_url'] = df_split['domain_url'].apply(get_domain)
df_split = df_split[df_split['domain_url'] != 'NA']
df_split.drop_duplicates(subset=['domain_url'], keep='last', inplace=True)
df_split

,domain_url,title,body_text,body_token,language,class
0,wwjewp6ca4rkudyj.onion,\n Frënn vun der Ënn - Index\n,[support FVDE because even small organizations...,"[support, fvde, becaus, even, small, organ, co...",en,Services
1,u76xgym22s4adf55.onion,PayPal Master: Home,[Questions Telegram us at httpsme ppmasteroffi...,"[question, telegram, u, httpsme, avail, respon...",en,Counterfeit
3,elherbotsiddarol.onion,ElHerbolario,"[Thanks, Fran ais Tutoriel, Listings price dro...","[thank, fran, ai, tutoriel, list, price, drop,...",en,Market
4,yermrrzeg4fusqx5.onion,anonymous OnionMail Server,"[To anonymous, anonymous OnionMail Server, Inf...","[anonym, anonym, onionmail, server, info, encr...",en,Services
5,safebtck4qejiero.onion,SafePay BTC | Bitcoin Escrow Payment Service,[If the seller does not deliver the order or i...,"[seller, doe, deliv, order, itunaccept, buyer,...",en,Cryptocurrency
...,...,...,...,...,...,...
10800,5wkkhgqtity7bhdf.onion,walletGenerator - Universal Paper wallet gener...,"[Firefox 21 may 2013, English, Add support for...","[firefox, may, english, add, support, alqo, co...",en,Cryptocurrency
10801,ctzzqqimlfamyhrc.onion,| TheYOSH.nl,[The last month the livecounter theyosh nl web...,"[last, month, livecount, theyosh, nl, websit, ...",en,Services
10802,grrmailb3fxpjbwm.onion,✉ Guerrilla Mail on Tor,"[Your Email Address is, Javascript is disabled...","[email, address, disabl, featur, avail, thank,...",en,Services
10803,greenegbqkyk3ois.onion,✔ Legit Carding Services ♛ Money Transfer Worl...,"[AmericanExpress PrePaid Cards, Hacked US Mast...","[americanexpress, prepaid, card, hack, u, mast...",en,Counterfeit


In [8]:
df_split.to_csv('../data/model_training_dataset_domain.csv', index=False)